# Interactive Narupa Client

This notebook assumes an IMD server is running, and lets you visualise it as a live trajectory with NGLView

In [1]:
from narupa.app.client import NarupaImdClient
from narupa.ase.converter import frame_data_to_ase
import numpy as np
import nglview

_ColormakerRegistry()

In [2]:
class NGLClient(NarupaImdClient):
    def __init__(self, dynamic_bonds=False, *args, update_callback=None, **kwargs):
        super().__init__(*args, **kwargs)
        self._view = None
        self.update_callback = update_callback
        self.dynamic_bonds = dynamic_bonds
    
    @property
    def view(self):
        if self._view is None or self.dynamic_bonds:
            atoms = frame_data_to_ase(
                client.first_frame,
                topology=True,
                positions=False,
            )
            atoms.set_positions(
                np.array(client.latest_frame.particle_positions) * 10
            )
            self._view = nglview.show_ase(atoms)
        return self._view

    def _on_frame_received(self, frame_index:int, frame):
        super()._on_frame_received(frame_index, frame)
        self.view.set_coordinates(
            {0: np.array(self.latest_frame.particle_positions) * 10}
        )
        if self.update_callback is not None:
            self.update_callback(self.universe)


def _get_structure_string(write_method, suffix='.pdb'):
    fname = 'tmp.pdb'
    write_method(fname)
    with open(fname) as fh:
        return fh.read()

nglview.adaptor._get_structure_string = _get_structure_string

In [3]:
client = NGLClient()

In [4]:
client.view

NGLWidget()